In [3]:
%autosave 60

Autosaving every 60 seconds


In [4]:
import os
import sys
import time
import requests
import datetime
import numpy as np
import pandas as pd
from dateutil.relativedelta import *
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [5]:
months = dict(zip(['августа',
 'апреля',  'декабря',  'июля',
 'июня',  'марта',  'мая',
 'ноября',  'октября',  'сентября',
 'февраля',  'января'], [8, 4, 12, 7, 6, 3, 5, 11, 10, 9, 2, 1]))

In [6]:
s=Service(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\98.0.4758.102\chromedriver.exe] found in cache


In [7]:
# !"{sys.executable}" -m pip install --upgrade webdriver_manager

In [8]:
# # Read already gathered data

# df_first_articles = pd.DataFrame()
# for i in os.listdir(os.getcwd()):
#     if i.find(".csv") > -1:
#         df_temp = pd.read_csv(i, sep=";")        
#         df_first_articles = pd.concat([df_first_articles, df_temp], sort=True)
# df_first_articles.columns=["num", "datestr", "type", "header", "text", "url"]
# df_first_articles = df_first_articles.sort_values(by=['num']).reset_index(drop=True)

## Часть 1: парсим новый сайт ЦБ
### Выгрузка пресс-релизов, новостей и выступлений
https://www.cbr.ru/

In [105]:
# Create two windows: one with the main site and one for sub-windows

url = "https://www.cbr.ru/"

driver_base = webdriver.Chrome(service=s)
driver_base.maximize_window()
driver_base.get(url)
parent_tab = driver_base.window_handles[0]

driver_runner1 = webdriver.Chrome(service=s)
driver_runner1.maximize_window()

In [106]:
# Собираем в 4 этапа: 
# Новости
# Пресс-релизы
# Интервью - не нужно
# Выступления

# Так как приходится кликать разделы вручную :(
# Запустить кликер -> считать категорию -> обновить страницу -> повторить

In [110]:
def parse_by_type(articles_headers, articles_links):
    counter = 0
    articles = []
    exception_articles = []
    for header, link in tqdm(zip(articles_headers, articles_links)):
        counter += 1
        try:
            article_type = header.text.split("\n")[-1]
            article_url = link.get_attribute(name='href')
            driver_runner1.get(article_url)
            article_date = driver_runner1.find_elements(By.TAG_NAME, value='.news-info-line_date')[0].text
            article_text = [a.text for a in driver_runner1.find_elements(By.TAG_NAME, value='p')]
            articles.append([counter, article_date, article_type, link.text, "\n".join(article_text).strip(), article_url])
        except Exception as e:
            print("Problem!")
            exception_articles.append((header, link))  
    return articles, exception_articles

In [207]:
%%time

# Clicker!
# Click all buttons "Загрузить ещё" to get all the news in one window

try:
    while True:
        buttons = driver_base.find_elements(By.TAG_NAME, value='#_buttonLoadNextEvt')
        button = [i for i in buttons if i.text == 'Загрузить еще']
        button[0].click()
except Exception as e:
    print(e)
    print("End reached!")

list index out of range
End reached!
Wall time: 1.44 s


In [108]:
news_headers = driver_base.find_elements(By.TAG_NAME, value='.news_info')
news_links = driver_base.find_elements(By.TAG_NAME, value='.news_title')
print(len(news_headers), len(news_links))

1020 1020


In [111]:
news_articles, news_exc = parse_by_type(news_headers, news_links)

df_news_new = pd.DataFrame(news_articles)
df_news_new = df_news_new[df_news_new[2] != ""].reset_index(drop=True)
df_news_new[0] = df_news_new[0] - 20

1020it [13:22,  1.27it/s]


In [153]:
pr_headers = driver_base.find_elements(By.TAG_NAME, value='.news_info')
pr_links = driver_base.find_elements(By.TAG_NAME, value='.news_title')
print(len(pr_headers), len(pr_links))

1020 1020


In [201]:
pr_articles, pr_exc = parse_by_type(pr_headers, pr_links)

df_pr_new = pd.DataFrame(pr_articles)
df_pr_new = df_pr_new[df_pr_new[2] != ""].reset_index(drop=True)
df_pr_new[0] = df_pr_new[0] - 20

In [205]:
a = df_pr_new.reset_index().iloc[720:]
a[a['index'] != a[0] -2]
df_pr_new.iloc[740:760]
# header, link = pr_exc[1]

# article_type = header.text.split("\n")[-1]
# article_url = link.get_attribute(name='href')
# driver_runner1.get(article_url)
# # article_date = driver_runner1.find_elements(By.TAG_NAME, value='.news-info-line_date')[0].text
# article_text = [a.text for a in driver_runner1.find_elements(By.TAG_NAME, value='p')]
# article_date = "11 марта 2020 года"

# df_pr_new = df_pr_new.append([[760, article_date, article_type, link.text, "\n".join(article_text).strip(), article_url]])
# df_pr_new = df_pr_new.sort_values(by=[0]).reset_index(drop=True)

In [208]:
speech_headers = driver_base.find_elements(By.TAG_NAME, value='.news_info')
speech_links = driver_base.find_elements(By.TAG_NAME, value='.news_title')
print(len(speech_headers), len(speech_links))

153 153


In [209]:
speech_articles, speech_exc = parse_by_type(speech_headers, speech_links)

153it [05:55,  2.32s/it]


In [213]:
df_speech_new = pd.DataFrame(speech_articles)
df_speech_new = df_speech_new[df_speech_new[2] != ""].reset_index(drop=True)
df_speech_new[0] = df_speech_new[0] - 20

In [ ]:
df_new = pd.DataFrame()
for df_articles_temp, name in zip([df_news_new, df_pr_new, df_speech_new], ["news","press_rel","speech"]):
    df_articles_temp.to_csv(f"{name}_new.csv", 
                            encoding="utf-8", sep=";", index=False)
    df_new = pd.concat([df_new, df_articles_temp])
df_new.columns=["num", "datestr", "type", "header", "text", "url"]
df_new = df_new.drop(columns=['num'])
df_new = df_new.reset_index(drop=True)

### Здесь можно забрать готовые данные

In [9]:
df_new = pd.DataFrame()
for name in ["news","press_rel","speech"]:
    df_articles_temp = pd.read_csv(f"{name}_new.csv", 
                            encoding="utf-8", sep=";")
    df_new = pd.concat([df_new, df_articles_temp])
df_new.columns=["num", "datestr", "type", "header", "text", "url"]
df_new = df_new.drop(columns=['num'])
df_new = df_new.reset_index(drop=True)

df_new['date'] = [f"{i[0]}-{months[i[1]]}-{i[2]}" for i in df_new.datestr.str.split().tolist()]
df_new['date'] = pd.to_datetime(df_new['date'], format='%d-%m-%Y')

In [10]:
df_new.head()

,datestr,type,header,text,url,date
0,21 февраля 2022 года,Новость,Объем средств на счетах эскроу в долевом строи...,"Объем средств, размещенных участниками долевог...",https://www.cbr.ru/press/event/?id=12705,2022-02-21
1,17 февраля 2022 года,Новость,Мониторинг отраслевых финансовых потоков: рост...,Динамика отраслевых финансовых потоков указыва...,https://www.cbr.ru/press/event/?id=12702,2022-02-17
2,17 февраля 2022 года,Новость,В январе инфляция ускорилась в большинстве рег...,Годовая инфляция в январе находилась в диапазо...,https://www.cbr.ru/press/event/?id=12701,2022-02-17
3,17 февраля 2022 года,Новость,Банк России разработал детскую игру о признака...,Игра «Космический детектив: тайна семи банкнот...,https://www.cbr.ru/press/event/?id=12698,2022-02-17
4,16 февраля 2022 года,Новость,"В 2021 году Банк России выявил почти 2,7 тысяч...",Среди них 871 финансовая пирамида (в 2020 году...,https://www.cbr.ru/press/event/?id=12695,2022-02-16


In [11]:
df_new[df_new.type == "Пресс-релиз"].min()

datestr                                  1 августа 2019 года
type                                             Пресс-релиз
header     А. Данилов назначен директором Департамента об...
text       1. В целях применения абзаца второго и третьег...
url        https://www.cbr.ru/press/pr/?file=01022021_084...
date                                     2019-07-17 00:00:00
dtype: object

## Часть 2: парсим старый сайт ЦБ
### Выгрузка пресс-релизов, новостей и выступлений
http://old.cbr.ru/press/

In [93]:
url = "http://old.cbr.ru/press/month_archive/press_archive/"

driver_base = webdriver.Chrome(service=s)
driver_base.maximize_window()
driver_base.get(url)
parent_tab = driver_base.window_handles[0]

driver_runner1 = webdriver.Chrome(service=s)
driver_runner1.maximize_window()

In [94]:
start_date = datetime.date(2010, 1, 1)
end_date = datetime.date(2019, 7, 16)

In [126]:
DETECTED = df.url.tolist() 

def half_parser(cur_date, end_date, counter):
    exc_articles = []
    good_articles = []
    
    driver_base.find_elements(By.TAG_NAME, value='.hasDatepicker')[0].send_keys("\b\b\b\b\b\b\b\b\b\b" + cur_date.strftime("%d.%m.%Y"))
    driver_base.find_elements(By.TAG_NAME, value='.hasDatepicker')[1].send_keys("\b\b\b\b\b\b\b\b\b\b" + end_date.strftime("%d.%m.%Y"))
    driver_base.find_elements(By.TAG_NAME, value='#UniDbQuery_searchbutton')[0].click()

    articles_dates = driver_base.find_elements(By.TAG_NAME, value='.source')
    articles_links = driver_base.find_elements(By.TAG_NAME, value='a')[15:-10]
    for article_date, link in tqdm(zip(articles_dates, articles_links)):
        if link.get_attribute(name='href') in DETECTED:
            continue
        counter += 1
        try:
            article_type = "Пресс-релиз"
            article_url = link.get_attribute(name='href')
            driver_runner1.get(article_url)
            article_text = [a.text for a in driver_runner1.find_elements(By.TAG_NAME, value='p')]
            good_articles.append([counter, article_date.text, article_type, link.text, "\n".join(article_text).strip(), article_url])
        except Exception as e:
            print("Problem!")
            exc_articles.append((article_date, link)) 
    return good_articles, exc_articles, counter


counter = 0
cur_date = start_date
temp_end = cur_date + relativedelta(months=+1) - relativedelta(days=+1)
exc_articles = []
good_articles = []
while temp_end < end_date:
    print(cur_date, " - ", temp_end)
    
    good_articles_temp, exc_articles_temp, counter = half_parser(cur_date, cur_date + relativedelta(days=+14), counter)
    good_articles += good_articles_temp
    exc_articles += exc_articles_temp
    
    good_articles_temp, exc_articles_temp, counter = half_parser(cur_date + relativedelta(days=+15), temp_end, counter)
    good_articles += good_articles_temp
    exc_articles += exc_articles_temp

    if cur_date.month == 2:
        pd.DataFrame(good_articles).to_csv(f"{cur_date}_old_all_arts.csv", encoding="utf-8", sep=";", index=False)    

    cur_date = cur_date + relativedelta(months=+1)
    temp_end = cur_date + relativedelta(months=+1) - relativedelta(days=+1)
    temp_end = min([temp_end, end_date])

2010-01-01  -  2010-01-31


21it [00:00, 108.25it/s]
54it [00:00, 108.47it/s]


2010-02-01  -  2010-02-28


61it [00:08,  7.31it/s] 
51it [00:00, 105.87it/s]


2010-03-01  -  2010-03-31


...

17it [00:00, 101.77it/s]
22it [00:00, 102.74it/s]


2019-05-01  -  2019-05-31


6it [00:00, 95.46it/s]
19it [00:00, 111.76it/s]


2019-06-01  -  2019-06-30


19it [00:00, 105.17it/s]
21it [00:00, 103.00it/s]


In [127]:
pd.DataFrame(good_articles).to_csv(f"old_all_arts.csv", encoding="utf-8", sep=";", index=False)    

In [33]:
df_pr_old = pd.read_csv("2020-02-01_old.csv",sep=";")

df_pr_old.columns=["num", "datestr", "type", "header", "text", "url"]
df_pr_old = df_pr_old.drop(columns=['num'])
df_pr_old = df_pr_old.reset_index(drop=True)

# df_pr_old['date'] = [f"{i[0]}-{months[i[1]]}-{i[2]}" for i in df_new.datestr.str.split().tolist()]
df_pr_old['date'] = pd.to_datetime(df_pr_old['datestr'], format='%d.%m.%Y')
df_pr_old.head()

,datestr,type,header,text,url,date
0,29.01.2010,Пресс-релиз,О размещении средств федерального бюджета на б...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100129_154621...,2010-01-29
1,29.01.2010,Пресс-релиз,О проведении аукциона по предоставлению кредит...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100129_154621...,2010-01-29
2,29.01.2010,Пресс-релиз,О предварительном графике проведения аукционов...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100129_154618...,2010-01-29
3,29.01.2010,Пресс-релиз,Об условиях проведения Банком России 01.02.201...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100129_105415...,2010-01-29
4,29.01.2010,Пресс-релиз,Об условиях проведения Банком России 29.01.201...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100129_104722...,2010-01-29


In [129]:
df_pr_old_extra = pd.DataFrame(good_articles)
df_pr_old_extra.columns=["num", "datestr", "type", "header", "text", "url"]
df_pr_old_extra = df_pr_old_extra.drop(columns=['num'])
df_pr_old_extra = df_pr_old_extra.reset_index(drop=True)

# df_pr_old['date'] = [f"{i[0]}-{months[i[1]]}-{i[2]}" for i in df_new.datestr.str.split().tolist()]
df_pr_old_extra['date'] = pd.to_datetime(df_pr_old_extra['datestr'], format='%d.%m.%Y')
df_pr_old_extra.head()

,datestr,type,header,text,url,date
0,03.02.2010,Пресс-релиз,Об условиях проведения Банком России 04.02.201...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100203_101623...,2010-02-03
1,03.02.2010,Пресс-релиз,Об условиях проведения Банком России 03.02.201...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100203_101620...,2010-02-03
2,03.02.2010,Пресс-релиз,О проведении депозитных аукционов,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100203_155532...,2010-02-03
3,02.02.2010,Пресс-релиз,Об условиях проведения Банком России 02.02.201...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100202_110217...,2010-02-02
4,02.02.2010,Пресс-релиз,Об условиях проведения Банком России 03.02.201...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100202_105712...,2010-02-02


## Объединяем часть 1 и 2

In [130]:
df = pd.concat([df_pr_old, df_new, df_pr_old_extra]).drop_duplicates().sort_values(by=['date', 'type']).reset_index(drop=True)
df = df[['date','type','header','text', 'url']]
df = df[(df.date.dt.date <= datetime.date(2022, 2, 1)) & (df.date.dt.date <= datetime.date(2022, 2, 1))]

In [142]:
df.to_csv("main.csv", encoding="utf-8", sep=";", index=False)    

In [131]:
l = [i.text for i in driver_base.find_elements(By.TAG_NAME, value='.referenceable')[1:] if i.text != "Архив выступлений"]
a = set(df[df.type == "Выступление"].header.tolist())

In [140]:
# df["rate"] = (df.type=="Пресс-релиз")&(df.header.str.find("ключев") > -1)
# df[(df.type=="Пресс-релиз")&(df.header.str.find("Банк России принял") > -1)&(df.header.str.find("ключ") > -1)]
# df[(df.type=="Пресс-релиз")&(df.header.str.find("Банк России сохранил ключевую ставку") > -1)]
# df[(df.type=="Пресс-релиз")&(df.header.str.find("О ключевой ставке Банка России") > -1)]
# df[(df.type=="Пресс-релиз")&(df.header.str.find("О системе процентных инструментов денежно-кредитной политики Банка России") > -1)]
df[(df.type=="Пресс-релиз")&(df.header.str.find("рефинансирования и") > -1)]

,date,type,header,text,url
153,2010-02-19,Пресс-релиз,О снижении ставки рефинансирования и процентны...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100219_111251...
274,2010-03-26,Пресс-релиз,О снижении ставки рефинансирования и процентны...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100326_113124...
395,2010-04-29,Пресс-релиз,О снижении ставки рефинансирования и процентны...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100429_112021...
494,2010-05-31,Пресс-релиз,О снижении ставки рефинансирования и процентны...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100531_172920...
614,2010-06-30,Пресс-релиз,О ставке рефинансирования и процентных ставках...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100630_110442...
740,2010-07-30,Пресс-релиз,О ставке рефинансирования и процентных ставках...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100730_113355...
956,2010-09-28,Пресс-релиз,О ставке рефинансирования и процентных ставках...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=100928_105457...
1088,2010-10-29,Пресс-релиз,О ставке рефинансирования и процентных ставках...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=101029_105043...
1198,2010-11-26,Пресс-релиз,Об уровне ставки рефинансирования и процентных...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=101126_105732...
1328,2010-12-24,Пресс-релиз,Об уровне ставки рефинансирования и процентных...,Департамент внешних и общественных связей Банк...,http://www.cbr.ru/press/PR/?file=101224_152719...
